## build custom circuits

In [16]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
import numpy as np
num_uncertainty_qubits = 3

model_register = QuantumRegister(num_uncertainty_qubits, 'model')
objective_register = QuantumRegister(1, 'objective')
objective_measurement_register = ClassicalRegister(1, 'objective_measurement')


test_circuit = QuantumCircuit(model_register, objective_register, objective_measurement_register)

num = 7

binary_num = bin(num)[2:].zfill(num_uncertainty_qubits)

step = (np.pi/4) / (2**num_uncertainty_qubits)
print(step)

for i in range(num_uncertainty_qubits):
    if binary_num[i] == '1':
        test_circuit.x(model_register[i])

for i in range(num_uncertainty_qubits):
    print(2**(num_uncertainty_qubits-i-1))
    test_circuit.cry(step*2**(num_uncertainty_qubits-i), model_register[i], objective_register[0])
        
test_circuit.measure(objective_register, objective_measurement_register)
test_circuit.draw()



0.09817477042468103
4
2
1


┌───┐                                     
                model_0: ┤ X ├─────■───────────────────────────────
                         ├───┤     │                               
                model_1: ┤ X ├─────┼──────────■────────────────────
                         ├───┤     │          │                    
                model_2: ┤ X ├─────┼──────────┼──────────■─────────
                         └───┘┌────┴────┐┌────┴────┐┌────┴─────┐┌─┐
              objective: ─────┤ Ry(π/4) ├┤ Ry(π/8) ├┤ Ry(π/16) ├┤M├
                              └─────────┘└─────────┘└──────────┘└╥┘
objective_measurement: 1/════════════════════════════════════════╩═
                                                                 0

In [18]:
from qiskit import Aer, execute

backend = Aer.get_backend('qasm_simulator')
job = execute(test_circuit, backend, shots=10000)
result = job.result()
counts = result.get_counts()
print(counts)

print(np.arcsin(np.sqrt(counts['1']/10000))/step)

{'1': 4077, '0': 5923}
7.05441667324314


/tmp/ipykernel_2483455/2897627190.py:4: DeprecationWarning: The function ``qiskit.execute_function.execute()`` is deprecated as of qiskit 0.46.0. It will be removed in the Qiskit 1.0 release. This function combines ``transpile`` and ``backend.run``, which is covered by ``Sampler`` :mod:`~qiskit.primitives`. Alternatively, you can also run :func:`.transpile` followed by ``backend.run()``.
  job = execute(test_circuit, backend, shots=10000)
